# Function Calling

This Notebook Covers the steps needed to implement Function calling capabilities with SambaNova models, leveraging Langchain Tools and Langchain integrations 

We also provide the [FunctionCallingLlm module](../src/function_calling.py) and the [usage notebook](./usage.ipynb), these can be used as a quick start to use function calling, and this notebook can be used as a guide for further customizing your function calling model and tools 

In [ ]:
# Import libraries
import os
import re
import sys
import json
import operator
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from typing import Optional, Union, Type, List
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import StructuredTool, ToolException, Tool, tool
from langchain_experimental.utilities import PythonREPL
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.globals import set_debug

set_debug(False)

# Get absolute paths for kit_dir and repo_dir
current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

# Adding directories to the Python module search path
sys.path.append(kit_dir)
sys.path.append(repo_dir)

from langchain_sambanova import ChatSambaNova

# load env variables from a .env file into Python environment 
load_dotenv(os.path.join(repo_dir, '.env'))

True

## Tools Definitions

Here, several langchain tools and custom tools are defined showing different ways of implementing them

### Basic tools

The most simple way of implementing a custom langchain tool is using the `@tool` decorator and defining your arguments schema 

#### Get time tool

In [2]:
# argument schema
# a sub-class of the BaseModel in Pydantic, provides information about the input arguments (type and description)
class GetTimeSchema(BaseModel):
    """Returns current date, current time or both."""

    kind: Optional[str] = Field(description='kind of information to retrieve "date", "time" or "both"')

In [3]:
# definition using @tool decorator
@tool(args_schema=GetTimeSchema)
def get_time(kind: str = 'both') -> str:
    """Returns current date, current time or both.

    Args:
        kind(str): date, time or both
    
    Returns:
        str: current date, current time or both
    """
    if kind == 'date':
        date = datetime.now().strftime('%d/%m/%Y')
        return f'Current date: {date}'
    elif kind == 'time':
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current time: {time}'
    else:
        date = datetime.now().strftime('%d/%m/%Y')
        time = datetime.now().strftime('%H:%M:%S')
        return f'Current date: {date}, Current time: {time}'

In [4]:
get_time.invoke({'kind': 'time'})

'Current time: 11:10:02'

In [5]:
get_time.get_input_schema().schema()

{'description': 'Returns current date, current time or both.',
 'properties': {'kind': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'description': 'kind of information to retrieve "date", "time" or "both"',
   'title': 'Kind'}},
 'required': ['kind'],
 'title': 'GetTimeSchema',
 'type': 'object'}

### Customized error handling tools

You can create a more complex tool that is able to raise specific tool error with some information, that can be send to the llm in order to fix the original call if its possible 

#### Calculator tool

In [6]:
# argument schema
class CalculatorSchema(BaseModel):
    """allow calculation of only basic operations: + - * and /
    with a string input expression"""

    expression: str = Field(..., description="expression to calculate, example '12 * 3'")

In [7]:
# function to use in the tool
def calculator(expression: str) -> Union[str, int, float]:
    """
    allow calculation of basic operations with a string input expression
    
    Args:
        expression (str): expression to calculate
    
    Returns:
        Union[str, int, float]: calculated value or error message
    """
    ops = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        'x': operator.mul,
        'X': operator.mul,
        '÷': operator.truediv,
        '/': operator.truediv,
    }
    tokens = re.findall(r'\d+\.?\d*|\+|\-|\*|\/|÷|x|X', expression)

    if len(tokens) == 0:
        raise ToolException(
            f"Invalid expression '{expression}', should only contain one of the following operators + - * x and ÷"
        )

    current_value = float(tokens.pop(0))

    while len(tokens) > 0:
        # The next token should be an operator
        op = tokens.pop(0)

        # The next token should be a number
        if len(tokens) == 0:
            raise ToolException(f"Incomplete expression '{expression}'")
        try:
            next_value = float(tokens.pop(0))

        except ValueError:
            raise ToolException('Invalid number format')

        except:
            raise ToolException('Invalid operation')

        # check division by 0
        if op in ['/', '÷'] and next_value == 0:
            raise ToolException('cannot divide by 0')

        current_value = ops[op](current_value, next_value)

    result = current_value

    return result


# tool error handler
def _handle_error(error: ToolException) -> str:
    return f'The following errors occurred during Calculator tool execution: `{error.args}`'


# tool definition
calculator = StructuredTool.from_function(
    func=calculator,
    args_schema=CalculatorSchema,
    handle_tool_error=_handle_error,  # set as True if you want the tool to trow a generic ToolError message "Tool execution error"
)

In [8]:
calculator.invoke('18*23.7 -5')

421.59999999999997

In [9]:
calculator.invoke('7 / 0')

"The following errors occurred during Calculator tool execution: `('cannot divide by 0',)`"

In [10]:
calculator.get_input_schema().schema()

{'description': 'allow calculation of only basic operations: + - * and /\nwith a string input expression',
 'properties': {'expression': {'description': "expression to calculate, example '12 * 3'",
   'title': 'Expression',
   'type': 'string'}},
 'required': ['expression'],
 'title': 'CalculatorSchema',
 'type': 'object'}

### Langchain Tools

(There are several built-in tools available in langchain library that can be directly used by the model, you can find a list of available tools [here](https://python.langchain.com/v0.1/docs/integrations/tools/) 

#### Python standard shell, or REPL (Read-Eval-Print Loop) tool

In [11]:
# argument schema
class ReplSchema(BaseModel):
    "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it."

    command: str = Field(..., description='python code to evaluate')

In [12]:
# tool definition
python_repl = PythonREPL()
python_repl = Tool(
    name='python_repl',
    description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
    func=python_repl.run,
    args_schema=ReplSchema,
)

In [13]:
python_repl.invoke({'command': 'for i in range(0,5):\n\tprint(i)'})

Python REPL can execute arbitrary code. Use with caution.


'0\n1\n2\n3\n4\n'

In [14]:
python_repl.get_input_schema().schema()

{'description': 'A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, you should print it out with `print(...)`, if you need a specific module you should import it.',
 'properties': {'command': {'description': 'python code to evaluate',
   'title': 'Command',
   'type': 'string'}},
 'required': ['command'],
 'title': 'ReplSchema',
 'type': 'object'}

#### SQL calling tool

This tool is a demonstration of how to implement an SQL query tool leveraging the build in QuerySQLDataBaseTool langchain tool, for this example we are passing the db schema to Llama3 8B model, but you can get better performance with your own fine tuned model.
>For fine tuning your own sql model go to [fine_tuning_sql kit](../../fine_tuning_sql/README.md)

In [16]:
# example sql query call
db_path = os.path.join(kit_dir, 'data/chinook.db')
db_uri = f'sqlite:///{db_path}'
db = SQLDatabase.from_uri(db_uri)
print(db.get_usable_table_names())
print(db.run('SELECT * FROM genres;'))

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']
[(1, 'Rock'), (2, 'Jazz'), (3, 'Metal'), (4, 'Alternative & Punk'), (5, 'Rock And Roll'), (6, 'Blues'), (7, 'Latin'), (8, 'Reggae'), (9, 'Pop'), (10, 'Soundtrack'), (11, 'Bossa Nova'), (12, 'Easy Listening'), (13, 'Heavy Metal'), (14, 'R&B/Soul'), (15, 'Electronica/Dance'), (16, 'World'), (17, 'Hip Hop/Rap'), (18, 'Science Fiction'), (19, 'TV Shows'), (20, 'Sci Fi & Fantasy'), (21, 'Drama'), (22, 'Comedy'), (23, 'Alternative'), (24, 'Classical'), (25, 'Opera')]


In [17]:
# argument schema
class QueryDBSchema(BaseModel):
    "A query generation tool. Use this to generate sql queries and retrieve the results from a database. Do not pass sql queries directly. Input must be a natural language question or instruction."

    query: str = Field(..., description='natural language question or instruction.')

In [ ]:
def sql_finder(text):
    """Search in a string for a SQL query or code with format"""

    # regex for finding sql_code_pattern with format:
    # ```sql
    #    <query>
    # ```
    sql_code_pattern = re.compile(r'```sql\s+(.*?)\s+```', re.DOTALL)
    match = sql_code_pattern.search(text)
    if match is not None:
        query = match.group(1)

        return query
    else:
        # regex for finding sql_code_pattern with format:
        # ```
        # <quey>
        # ```
        code_pattern = re.compile(r'```\s+(.*?)\s+```', re.DOTALL)
        match = code_pattern.search(text)
        if match is not None:
            query = match.group(1)
            return query
        else:
            print(text)
            raise Exception('No SQL code found in LLM generation')


@tool(args_schema=QueryDBSchema)
def query_db(query: str) -> str:
    """A query generation tool. Use this to generate sql queries and retrieve the results from a database. 
    Do not pass sql queries directly. Input must be a natural language question or instruction.
    
    Args:
        query (src): natural language question or instruction
    
    Returns:
        str: result from the database
    """

    # Using SambaNova model for generating the SQL Query
    llm = ChatSambaNova(
        base_url="https://api.sambanova.ai/v1",
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        max_tokens=512,
        model='Meta-Llama-3.3-70B-Instruct',
    )

    prompt = ChatPromptTemplate(
        [
            (
                'system',
                """
            {table_info}
            
            Generate a query using valid SQLite to answer the following questions for the summarized tables schemas provided above.
            Do not assume the values on the database tables before generating the SQL query, always generate a SQL that query what is asked.
            Do not assume ids in tables when inserting new values let them null or use the max id + 1
            The queries must be formatted including backticks code symbols as follows:
            do not include comments in the query
                
            ```sql
            query
            ```
            
            Example format:
            
            ```sql
            SELECT * FROM mainTable;
            ```""",  # noqa: E501
            ),
            ('human', """{input}"""),
        ]
    )

    # Chain that receives the natural language input and the table schema, then pass the teh formmated prompt to the llm
    # and finally execute the sql finder method, retrieving only the filtered SQL query
    query_generation_chain = prompt | llm | StrOutputParser() |RunnableLambda(sql_finder)
    table_info = db.get_table_info()
    query = query_generation_chain.invoke({'input': query, 'table_info': table_info})
    queries = query.split(';')
    query_executor = QuerySQLDataBaseTool(db=db)
    results = []
    for query in queries:
        if query.strip() != '':
            print(f'query_db: executing query: \n{query}\n')
            results.append(query_executor.invoke(query))
            print(f'query_db: query result: \n{results[-1]}\n')

    result = '\n'.join([f'Query {query} executed with result {result}' for query, result in zip(queries, results)])

    return result

##### Examples of query_db tool call

In [30]:
query_db.invoke({'query': 'How many genres of music are in the chinook db'})

query_db: executing query: 
SELECT COUNT(*) FROM genres

query_db: query result: 
[(25,)]



'Query SELECT COUNT(*) FROM genres executed with result [(25,)]'

In [31]:
query_db.invoke({'query': 'add a new genre in the chinook db called Salsa'})

query_db: executing query: 
INSERT INTO genres (Name) 
VALUES ('Salsa')

query_db: query result: 




"Query INSERT INTO genres (Name) \nVALUES ('Salsa') executed with result "

In [32]:
query_db.invoke({'query': 'What is the longest track in the chinook db'})

query_db: executing query: 
SELECT Name, MAX(Milliseconds) as Duration 
FROM tracks

query_db: query result: 
[('Occupation / Precipice', 5286953)]



"Query SELECT Name, MAX(Milliseconds) as Duration \nFROM tracks executed with result [('Occupation / Precipice', 5286953)]"

In [33]:
query_db.invoke({'query': 'list of the 5 highest value invoices registered in the database including customer names'})

query_db: executing query: 
SELECT i.InvoiceId, i.Total, c.FirstName, c.LastName 
FROM invoices i 
JOIN customers c ON i.CustomerId = c.CustomerId 
ORDER BY i.Total DESC 
LIMIT 5

query_db: query result: 
[(404, 25.86, 'Helena', 'Holý'), (299, 23.86, 'Richard', 'Cunningham'), (96, 21.86, 'Ladislav', 'Kovács'), (194, 21.86, 'Hugh', "O'Reilly"), (89, 18.86, 'Astrid', 'Gruber')]



'Query SELECT i.InvoiceId, i.Total, c.FirstName, c.LastName \nFROM invoices i \nJOIN customers c ON i.CustomerId = c.CustomerId \nORDER BY i.Total DESC \nLIMIT 5 executed with result [(404, 25.86, \'Helena\', \'Holý\'), (299, 23.86, \'Richard\', \'Cunningham\'), (96, 21.86, \'Ladislav\', \'Kovács\'), (194, 21.86, \'Hugh\', "O\'Reilly"), (89, 18.86, \'Astrid\', \'Gruber\')]'

In [34]:
query_db.invoke(
    {
        'query': 'add in the db a new song called "El Preso" from the artist "Fruco y sus tesos", with a duration 123000 seconds with a price of 1.2 usd'
    }
)

query_db: executing query: 
INSERT INTO artists ("Name") 
VALUES ('Fruco y sus tesos')

query_db: query result: 


query_db: executing query: 


INSERT INTO albums ("Title", "ArtistId") 
VALUES ('Fruco y sus tesos', (SELECT MAX("ArtistId") FROM artists))

query_db: query result: 


query_db: executing query: 


INSERT INTO tracks ("Name", "AlbumId", "MediaTypeId", "Milliseconds", "UnitPrice") 
VALUES ('El Preso', (SELECT MAX("AlbumId") FROM albums), 1, 123000, 1.2)

query_db: query result: 




'Query INSERT INTO artists ("Name") \nVALUES (\'Fruco y sus tesos\') executed with result \nQuery \n\nINSERT INTO albums ("Title", "ArtistId") \nVALUES (\'Fruco y sus tesos\', (SELECT MAX("ArtistId") FROM artists)) executed with result \nQuery \n\nINSERT INTO tracks ("Name", "AlbumId", "MediaTypeId", "Milliseconds", "UnitPrice") \nVALUES (\'El Preso\', (SELECT MAX("AlbumId") FROM albums), 1, 123000, 1.2) executed with result '

In [35]:
query_db.invoke({'query': 'who is the composer of the song with name "El Preso"'})

query_db: executing query: 
SELECT Composer FROM tracks WHERE Name = 'El Preso'

query_db: query result: 
[(None,)]



"Query SELECT Composer FROM tracks WHERE Name = 'El Preso' executed with result [(None,)]"

In [36]:
query_db.invoke(
    {'query': 'create a new table in the chinook db called movies, with a column for id and column for movieTitle'}
)

query_db: executing query: 
CREATE TABLE movies (
    `MovieId` INTEGER PRIMARY KEY,
    `MovieTitle` NVARCHAR(200) NOT NULL
)

query_db: query result: 




'Query CREATE TABLE movies (\n    `MovieId` INTEGER PRIMARY KEY,\n    `MovieTitle` NVARCHAR(200) NOT NULL\n) executed with result '

In [37]:
# for exectuing this query it is needed to reinitialize the query db to get the new schema including movies table
query_db.invoke({'query': 'add a new register in movies table for "star wars"'})

query_db: executing query: 
INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING

query_db: query result: 
Error: (sqlite3.OperationalError) ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint
[SQL: INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

query_db: executing query: 

INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING

query_db: query result: 
Error: (sqlite3.OperationalError) ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint
[SQL: 
INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

query_db: executing query: 


INSERT INTO playlists (Name) 
SELECT 'Movies'
WHERE NOT EXISTS (
  SELECT 1 
  FROM playlists 
  WHERE Name = 'Movies'
)

query_db: query result: 


query_db: executing query: 


INSERT INTO playlist_trac

"Query INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING executed with result Error: (sqlite3.OperationalError) ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint\n[SQL: INSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)\nQuery \nINSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING executed with result Error: (sqlite3.OperationalError) ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint\n[SQL: \nINSERT INTO playlists (Name) VALUES ('Movies') ON CONFLICT (Name) DO NOTHING]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)\nQuery \n\nINSERT INTO playlists (Name) \nSELECT 'Movies'\nWHERE NOT EXISTS (\n  SELECT 1 \n  FROM playlists \n  WHERE Name = 'Movies'\n) executed with result \nQuery \n\nINSERT INTO playlist_track (PlaylistId, TrackId) \nSELECT p.PlaylistId, (SELECT MAX(t.TrackId) + 1 FROM tracks 

### Default response tool

We define a default tool to return conversational responses to the user, for this we will only define the schema of the tool and manually handle the behavior of the system when this tool is called by the model

In [38]:
# argument schema
class ConversationalResponse(BaseModel):
    "Respond conversationally only if no other tools should be called for a given query, or if you have a final answer. response must be in the same language as the user query"

    response: str = Field(
        ..., description='Conversational response to the user. must be in the same language as the user query'
    )


ConversationalResponse.schema()

{'description': 'Respond conversationally only if no other tools should be called for a given query, or if you have a final answer. response must be in the same language as the user query',
 'properties': {'response': {'description': 'Conversational response to the user. must be in the same language as the user query',
   'title': 'Response',
   'type': 'string'}},
 'required': ['response'],
 'title': 'ConversationalResponse',
 'type': 'object'}

In [39]:
def get_tools_schemas(
    tools: Optional[Union[StructuredTool, Tool, list]] = None,
    default: Optional[Union[StructuredTool, Tool, Type[BaseModel]]] = None,
) -> list:
    """
    Get the tools schemas.

    Args:
        tools (Optional[Union[StructuredTool, Tool, list]]): The tools to use.
        default (Optional[Union[StructuredTool, Tool, Type[BaseModel]]]): The default tool to use.
    
    Returns:
        list: list of tool schemas  
    """
    if tools is None or isinstance(tools, list):
        pass
    elif isinstance(tools, Tool) or isinstance(tools, StructuredTool):
        tools = [tools]
    else:
        raise TypeError('tools must be a Tool or a list of Tools')

    tools_schemas = []

    for tool in tools:
        tool_schema = tool.get_input_schema().schema()
        schema = {'name': tool.name, 'description': tool_schema['description'], 'properties': tool_schema['properties']}
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    if default is not None:
        if isinstance(default, Tool) or isinstance(default, StructuredTool):
            tool_schema = default.get_input_schema().schema()
        elif issubclass(default, BaseModel):
            tool_schema = default.schema()
        else:
            raise TypeError('default must be a Tool or a BaseModel')
        schema = {
            'name': tool_schema['title'],
            'description': tool_schema['description'],
            'properties': tool_schema['properties'],
        }
        if 'required' in schema:
            schema['required'] = tool_schema['required']
        tools_schemas.append(schema)

    return tools_schemas

#### Set of tools

In [40]:
tools = [get_time, calculator, python_repl, query_db]

In [41]:
# Get list of tools schemas
tools_schemas = get_tools_schemas(tools, default=ConversationalResponse) 

# Convert list of tools schemas into JSON string
tools_schemas = '\n'.join([json.dumps(tool, indent=2) for tool in tools_schemas])
pprint(tools_schemas)

('{\n'
 '  "name": "get_time",\n'
 '  "description": "Returns current date, current time or both.",\n'
 '  "properties": {\n'
 '    "kind": {\n'
 '      "anyOf": [\n'
 '        {\n'
 '          "type": "string"\n'
 '        },\n'
 '        {\n'
 '          "type": "null"\n'
 '        }\n'
 '      ],\n'
 '      "description": "kind of information to retrieve \\"date\\", \\"time\\" '
 'or \\"both\\"",\n'
 '      "title": "Kind"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "calculator",\n'
 '  "description": "allow calculation of only basic operations: + - * and '
 '/\\nwith a string input expression",\n'
 '  "properties": {\n'
 '    "expression": {\n'
 '      "description": "expression to calculate, example \'12 * 3\'",\n'
 '      "title": "Expression",\n'
 '      "type": "string"\n'
 '    }\n'
 '  }\n'
 '}\n'
 '{\n'
 '  "name": "python_repl",\n'
 '  "description": "A Python shell. Use this to evaluate python commands. '
 'Input should be a valid python commands and expressions. If y

## Function Calling Chain

Workflow for tool execution

### LLM definition

In [ ]:
# Using SambaNova model for tool calling
llm = ChatSambaNova(
    max_tokens=2048,
    model="Meta-Llama-3.3-70B-Instruct",
)

### Tool execution

Definition of some util methods to parse the llm output and invoke available tools

In [43]:
# map tool name to tool
tools_map = {'get_time': get_time, 'calculator': calculator, 'python_repl': python_repl, 'query_db': query_db}


def execute(invoked_tools: List[dict]) -> tuple[bool, List[str]]:
    """
    Executes a list of tool invocations as generated by the LLM. Each tool is executed
    according to its name, which is mapped in tools_map, along with the provided input arguments.
    If there is only one tool invocation and it is the default conversational tool, the response is marked as the final response.
    
    Args:
        invoked_tools (List[dict]): A list of tool invocations as generated by the LLM.
    
    Returns:
        tuple[bool, List[str]]: A tuple containing:
            - A boolean indicating whether the response is marked as the final response
            - A list of strings representing the results from the executed tools
    """
    tool_msg = "Tool '{name}'response: {response}"
    tools_msgs = []
    if len(invoked_tools) == 1 and invoked_tools[0]['tool'].lower() == 'conversationalresponse':
        final_answer = True
        return final_answer, [invoked_tools[0]['tool_input']['response']]
    for tool in invoked_tools:
        final_answer = False
        if tool['tool'].lower() != 'conversationalresponse':
            response = tools_map[tool['tool'].lower()].invoke(tool['tool_input'])
            tools_msgs.append(tool_msg.format(name=tool['tool'], response=str(response)))
    return final_answer, tools_msgs

In [44]:
def jsonFinder(input_message: AIMessage) -> Optional[str]:
    """
    Searches for JSON structures within a LLM response. If the JSON is badly formatted, it attempts to correct it using an LLM.

    Args:
        input_message (AIMessage): The message to find the JSON structure in.
    
    Returns:
        Optional[str]: JSON formatted response
    """
    json_pattern = re.compile(r'(\{.*\}|\[.*\])', re.DOTALL)
    # Find the first JSON structure in the string
    json_match = json_pattern.search(input_message.content)
    if json_match: # Case when a JSON structure is found
        json_str = json_match.group(1)
        try:
            json.loads(json_str)
        except:
            json_correction_prompt = [
                ('system', """You are a json format corrector tool"""),
                ('human', """fix the following json file: {json}
                 do not provide any explanation only return the fixed json""")
            ]
            json_correction_prompt_template = ChatPromptTemplate(json_correction_prompt)
            json_correction_chain = json_correction_prompt_template | llm | StrOutputParser()
            json_str = json_correction_chain.invoke(json_str)
    else: # Case when no JSON structure is found
        # will assume its a conversational response
        print('response is not json formatted assuming conversational response')
        dummy_json_response = [{'tool': 'ConversationalResponse', 'tool_input': {'response': input_message.content}}]
        json_str = json.dumps(dummy_json_response)
    return json_str

Agentic prompt template for function calling

In [45]:
example_function_calling_prompt = [
  ('system',"""You are an helpful assistant and you have access to the following tools:

  {tools}

  You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

  ```json
  [{{
    "tool": <name of the selected tool>,
    "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
  }}]
  ```

  Think step by step
  Do not call a tool if the input depends on another tool output you dont have yet.
  Do not try to answer until you get tools output, if you dont have an answer yet you can continue calling tools until you do.
  Your answer should be in the same language as the initial query."""),
  ('human',"""User: {usr_msg} """)
]

function_calling_prompt_template = ChatPromptTemplate(example_function_calling_prompt)

Json parsing Chain for parsing tool calling output from llm

In [46]:
json_parsing_chain = RunnableLambda(jsonFinder) | JsonOutputParser()

Chain for passing through the tools schemas to the model

In [47]:
prompt_template = {
    'tools': lambda x: tools_schemas,
    'usr_msg': RunnablePassthrough(),
} | function_calling_prompt_template

Example of chain to do one full pass of the query to the model. 
Here we will see the first tool execution call generated by the model and the result of this first execution 

In [48]:
# define function calling chain
default_fc_chain = prompt_template | llm | json_parsing_chain

In [49]:
query = 'hi'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'ConversationalResponse', 'tool_input': {'response': 'Hello! How can I help you today?'}}]
['Hello! How can I help you today?']


In [50]:
query = 'it is time to go to sleep?'
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'get_time', 'tool_input': {'kind': 'time'}}]
["Tool 'get_time'response: Current time: 12:50:39"]


In [51]:
query = 'what is 347 min in hours and minutes?'
response_tools = default_fc_chain.invoke(query)
print(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'calculator', 'tool_input': {'expression': '347 / 60'}}]
["Tool 'calculator'response: 5.783333333333333"]


In [52]:
query = "is this word is a palindrome? 'saippuakivikauppias'"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print('saippuakivikauppias' == "
                            "'saippuakivikauppias'[::-1])"}}]
["Tool 'python_repl'response: True\n"]


In [53]:
query = "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']"
response_tools = default_fc_chain.invoke(query)
pprint(response_tools)
final_answer, response_tools = execute(response_tools)
print(response_tools)

[{'tool': 'python_repl',
  'tool_input': {'command': "print(sorted(['screwdriver', 'pliers', "
                            "'hammer']))"}}]
["Tool 'python_repl'response: ['hammer', 'pliers', 'screwdriver']\n"]


### Function Calling pipeline 

Here we are defining a iterative pipeline to give the model the ability of getting the tools execution as inputs and continue generating the answer until having a final response 

Definition of the system message

In [54]:
function_calling_system_prompt = """you are an helpful assistant and you have access to the following tools:

{tools}

You must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:

```json
[{{
  "tool": <name of the selected tool>,
  "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
}}]
```

Think step by step
Do not call a tool if the input depends on another tool output that you do not have yet.
Do not try to answer until you get all the tools output, if you do not have an answer yet, you can continue calling tools until you do.
Your answer should be in the same language as the initial query.

"""

Creation of chat prompt template, having as first interaction the system prompt

In [55]:
function_calling_chat_template = ChatPromptTemplate.from_messages([('system', function_calling_system_prompt)])
function_calling_chat_template

ChatPromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='you are an helpful assistant and you have access to the following tools:\n\n{tools}\n\nYou must always select one or more of the above tools and answer with only a list of JSON objects matching the following schema:\n\n```json\n[{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}]\n```\n\nThink step by step\nDo not call a tool if the input depends on another tool output that you do not have yet.\nDo not try to answer until you get all the tools output, if you do not have an answer yet, you can continue calling tools until you do.\nYour answer should be in the same language as the initial query.\n\n'), additional_kwargs={})])

Putting everything together to have an iterative/agentic pipeline for doing function calling

In [58]:
def function_call_llm(query: str, max_it: int = 5, debug: bool = False) -> str:
    """
    Iterative/agentic pipeline for function calling workflow

    Args:
        query (str): The query to execute.
        max_it (int, optional): The maximum number of iterations. Defaults to 5.
        debug (bool, optional): Whether to print debug information. Defaults to False.

    Returns:
        str: The final response from the LLM or an exception if no final response is reached.
    """
    history = function_calling_chat_template.format_prompt(tools=tools_schemas).to_messages()
    history.append(HumanMessage(query))
    tool_call_id = 0  # Unique identifier for each tool calling required to create ToolMessages

    for i in range(max_it):
        llm_response = llm.invoke(history)
        parsed_tools_llm_response = json_parsing_chain.invoke(llm_response)
        history.append(llm_response)
        final_answer, tools_msgs = execute(parsed_tools_llm_response)
        if final_answer:
            final_response = tools_msgs[0]
            if debug:
                pprint(history)
            return final_response
        else:
            history.append(ToolMessage('\n'.join(tools_msgs), tool_call_id=tool_call_id))
            tool_call_id += 1

    # If no final response is reached after the maximum number of iterations 
    raise Exception('not a final response yet', json.dumps(history))

In [61]:
response = function_call_llm('what time is it?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, 

In [62]:
print(response)

It's 12:53:58


In [63]:
response = function_call_llm('how many hours last to 10pm?', max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, 

In [64]:
response

'there are 10 hours until 10pm'

In [65]:
response = function_call_llm("is this word is a palindrome? 'saippuakivikauppias'", max_it=5, debug=True)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, 

In [66]:
response

"Yes, the word 'saippuakivikauppias' is a palindrome."

In [67]:
response = function_call_llm(
    "sort this list of elements alphabetically ['screwdriver', 'pliers', 'hammer']", max_it=5, debug=True
)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, 

In [68]:
response

"['hammer', 'pliers', 'screwdriver']"

In [69]:
response = function_call_llm(
    'whats the price in colombian pesos of the track "Snowballed" in the db if one usd is equal to 3800 cop?',
    max_it=5,
    debug=True,
)

query_db: executing query: 
SELECT UnitPrice FROM tracks WHERE Name = 'Snowballed'

query_db: query result: 
[(0.99,)]

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evalu

In [70]:
response

"The price of the track 'Snowballed' in Colombian Pesos is 3762.0 COP."

Instructions in other languages

In [71]:
response = function_call_llm(
    '¿cuantos días faltan para navidad?',
    max_it=5,
    debug=True,
)

[SystemMessage(content='you are an helpful assistant and you have access to the following tools:\n\n{\n  "name": "get_time",\n  "description": "Returns current date, current time or both.",\n  "properties": {\n    "kind": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "description": "kind of information to retrieve \\"date\\", \\"time\\" or \\"both\\"",\n      "title": "Kind"\n    }\n  }\n}\n{\n  "name": "calculator",\n  "description": "allow calculation of only basic operations: + - * and /\\nwith a string input expression",\n  "properties": {\n    "expression": {\n      "description": "expression to calculate, example \'12 * 3\'",\n      "title": "Expression",\n      "type": "string"\n    }\n  }\n}\n{\n  "name": "python_repl",\n  "description": "A Python shell. Use this to evaluate python commands. Input should be a valid python commands and expressions. If you want to see the output of a value, 

In [72]:
response

'Faltan 63 días para Navidad.'